To get more efficient algorithm for solving the game we need to better identify when a ship has been sunk. Thankfully, the official rules of the game help us in this regard. Up until now, we have only used two states for giving feedback on each shot: **HIT** and **MISS**.

The official rules of the game also state that you should let your opponent know if they have successfully **SUNK** any ship, so this third style message "*You have sunk my aircraft carrier*" conveys much more information than just hit.

It tells you the length of the ship you have just hit, it tells you that you have hit all pixels of this ship, and it could, potentially, give you a new minimum size of ship you are searching for (for instance, if you have sunk all ships other than the aircraft carrier, then you know that this remaining ship is five units long and can adjust your random search according to skip the appropriate number of spaced when in hunt mode.

First thing to implement is adding some way of knowing if ship is sunk. Currently, there's no notion of a ship. Gameboard just draws squares on passed locations. We'll try to do this without completely rewriting existing classes. For that, we make a Ship class. But we add generate_separate_locations method to Random class first. That method will help us initialize ship objects.

In [1]:
import random as pyrandom
from Gameboard import *

In [2]:
class Random:
    def __init__(self):
        # possible locations list isn't affected by the methods
        self.possible_locations = []
        self.available_locations = []
        for i in Gameboard.row_labels:
            for j in Gameboard.col_labels:
                self.possible_locations.append(i + str(j))
                self.available_locations.append(i + str(j))
        
    def reset_available_locations(self):
        self.available_locations = self.possible_locations.copy()
        
    # chooses random location from available locations list and removes it if keep is False
    def choose(self, keep=False):
        choice = pyrandom.choice(self.available_locations)
        if not keep:
            self.available_locations.remove(choice)
        return choice
    
    # returns random location from possible locations list
    def choose_from_all(self):
        return pyrandom.choice(self.possible_locations)
    
    # removes passed location from available locations list 
    def make_unavailable(self, location):
        self.available_locations.remove(location)
    
    # generates random locations for ships of various length and returns them
    # WARGING: current implementation is not optimal and should be changed sometime in the future
    # NOTE: this method currently uses choose method but it resets available locations at the end
    def generate_ship_locations(self, lengths=[5, 4, 3, 3, 2]):
        flat_list = []
        for sublist in self.generate_separate_locations(lengths):
            for item in sublist:
                flat_list.append(item)
        return flat_list
    
    # returns array of ship locations, where ship locations are array of locations themselves
    # for example: [2, 3] returns [['A1', 'A2'], ['C3', 'C4', 'C5']]
    def generate_separate_locations(self, lengths=[5, 4, 3, 3, 2]):
        locations = []
        
        for length in lengths:
            orientation = pyrandom.choice(['horizontal', 'vertical'])
            
            # randomize a ship location until it fits
            while True:
                choice_pivot = self.choose(keep=True)
                choices = [choice_pivot]
                if orientation == 'horizontal':   
                    # generate all other choices (for horizontal orientation)
                    for i in range(length - 1):
                        # when orientation is horizontal, column is changed
                        choice_next = choice_pivot[0] + str(int(choice_pivot[1:]) + i + 1)
                        choices.append(choice_next)
                else:
                    # generate all other choices (for vertical orientation)
                    for i in range(length - 1):
                        # when orientation is vertical, row is changed
                        choice_next = chr(ord(choice_pivot[0]) + i + 1) + choice_pivot[1:]
                        choices.append(choice_next)
                # after generation, we must check if locations are legal
                is_legal = True
                for choice in choices:
                    if choice not in self.available_locations:
                        is_legal = False
                # if they're legal, we update available locations and break the loop
                if is_legal:
                    self.available_locations = [x for x in self.available_locations if x not in choices]
                    locations.append(choices)
                    break
        
        # reset available locations and return result
        self.reset_available_locations()
        return locations

In [3]:
# test newly added method
random = Random()
random.generate_separate_locations()

[['I5', 'I6', 'I7', 'I8', 'I9'],
 ['G3', 'H3', 'I3', 'J3'],
 ['A5', 'B5', 'C5'],
 ['B1', 'C1', 'D1'],
 ['A4', 'B4']]

In [4]:
class Ship:
    # Random class already can generate locations, so we can initialize ship after that
    def __init__(self, locations):
        self.locations = locations
        self.length = len(locations)
        
    def get_length(self):
        return self.length
    
    def get_locations(self):
        return self.locations
    
    # TODO: there are more methods to add...        

In [5]:
ship_1 = Ship(random.generate_separate_locations()[0])
print(ship_1.get_locations())
print(ship_1.get_length())

['B4', 'B5', 'B6', 'B7', 'B8']
5
